In [1]:
import os
import cv2
import numpy as np
from mss import mss
from PIL import Image
import pytesseract
import pyperclip

TESSERACT_PATH = r"C:\Pyg\Program_Files\Tesseract-OCR\tesseract.exe"

if os.path.exists(TESSERACT_PATH):
    pytesseract.pytesseract.tesseract_cmd = TESSERACT_PATH

def capture_fullscreen_bgr():
    """전체 화면을 캡처해서 OpenCV(BGR) 이미지로 반환"""
    with mss() as sct:
        monitor = sct.monitors[1]  # 기본 주 모니터
        img = np.array(sct.grab(monitor))  # BGRA
        bgr = cv2.cvtColor(img, cv2.COLOR_BGRA2BGR)
        return bgr

def ocr_to_text(bgr_img, lang="kor+eng"):
    """BGR 이미지를 OCR해서 텍스트 반환"""
    gray = cv2.cvtColor(bgr_img, cv2.COLOR_BGR2GRAY)

    # 가벼운 전처리(가독성 향상)
    gray = cv2.GaussianBlur(gray, (3, 3), 0)
    _, thr = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    config = r"--oem 3 --psm 6"
    text = pytesseract.image_to_string(thr, lang=lang, config=config)
    return text.strip()

def copy_text_to_clipboard(text: str):
    pyperclip.copy(text)
    return text


In [ ]:
def run_capture_crop_ocr(lang="kor+eng"):
    screen = capture_fullscreen_bgr()

    # ROI 선택 UI (드래그해서 영역 선택 후 ENTER / 취소는 ESC)
    roi = cv2.selectROI("Select ROI (ENTER=OK, ESC=Cancel)", screen, showCrosshair=True, fromCenter=False)
    cv2.destroyAllWindows()

    x, y, w, h = roi
    if w == 0 or h == 0:
        print("❌ ROI 선택이 취소되었거나 크기가 0입니다.")
        return ""

    cropped = screen[y:y+h, x:x+w]
    text = ocr_to_text(cropped, lang=lang)

    copy_text_to_clipboard(text)
    print("✅ OCR 완료 + 클립보드 복사 완료\n")
    print(text)
    return text

# 실행
_ = run_capture_crop_ocr(lang="kor+eng")

In [ ]:
# 디버그용 출력
print("tesseract cmd:", getattr(pytesseract.pytesseract, "tesseract_cmd", None))
print("clipboard test:", pyperclip.paste()[:200])